## Business Central merge data notebook
In this part the files in the delta folder will be merge with the Lakehouse table.
- It iterates first on the folders to append to the existing table.
- After that is will remove all duplicates by sorting the table. 
- At last it will remove all deleted records inside the table that are deleted in Business Central

Please change the parameters in the first part.

In [11]:
%%pyspark
# settings
spark.conf.set("sprk.sql.parquet.vorder.enabled","true")
spark.conf.set("spark.microsoft.delta.optimizewrite.enabled","true")
spark.conf.set("spark.sql.parquet.filterPushdown", "true")
spark.conf.set("spark.sql.parquet.mergeSchema", "false")
spark.conf.set("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
spark.conf.set("spark.sql.delta.commitProtocol.enabled", "true")

# parameters
folder_path_spark = 'Files/deltas/' # this is mostly the default
folder_path = '/lakehouse/default/Files/deltas/' # this is mostly the default
workspace = 'fabricTest' #can also be a GUID
Lakehouse = 'businessCentral'; #can also be a GUID
Remove_delta = True;

StatementMeta(, daf5b681-1577-45ce-aec5-3696d3564aa0, 13, Finished, Available)

In [12]:
%%pyspark
import json
import os
import glob
from pyspark.sql.types import *
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

for entry in os.scandir(folder_path):
 if entry.is_dir():
    if glob.glob(folder_path + entry.name + '/*'):
        table_name = entry.name.replace("-","")

        df_new = spark.read.format("csv").option("header","true").load(folder_path_spark + entry.name +"/*")   
        
        f = open("/lakehouse/default/Files/"+ entry.name +".cdm.json")
        schema = json.load(f)
        # Parse the schema to get column names and data types
        column_names = [attr["name"] for attr in schema["definitions"][0]["hasAttributes"]] 
        column_types = [attr['dataFormat'] for attr in schema["definitions"][0]["hasAttributes"]]   
        for col_name, col_type in zip(column_names, column_types):
            if col_type == "String":
                col_type = "string"
            if col_type == "Guid":
                col_type = "string"
            if col_type == "Code":
                col_type = "object"
            if col_type == "Option":
                col_type = "string"
            if col_type == "Date":
                col_type = "date"
            if col_type == "DateTime":
                col_type = "date"
            if col_type == "Duration":
                col_type = "timedelta"
            if col_type == "Decimal":
                col_type = "float"
            if col_type == "Boolean":
                col_type = "boolean"
            if col_type == "Integer":
                col_type = "int"
            if col_type == "Int64":
                col_type = "int"
            if col_type == "Int32":
                col_type = "int"

            df_new = df_new.withColumn(col_name, df_new[col_name].cast(col_type))

        #check if the table excists
        if table_name in [t.name for t in spark.catalog.listTables()]:  
            #read the old data into a new dataframe and union with the new dataframe
            SQL_Query = "SELECT * FROM " + Lakehouse +"."+table_name;  
            df_old = spark.sql(SQL_Query)
            df_new = df_new.union(df_old)

            #delete all old records
            df_deletes = df_new.filter(df_new['SystemCreatedAt-2000000001'].isNull())
            df_new = df_new.join(df_deletes, ["systemId-2000000000"], "leftanti")

            # remove duplicates by filtering on systemID and systemModifiedAt fields
            df_new = df_new.orderBy('systemId-2000000000',desc('SystemModifiedAt-2000000003'))
            df_new = df_new.dropDuplicates(['systemId-2000000000'])

            #overwrite the dataframe in the new table
            df_new.write.mode("overwrite").format("delta").save("Tables/" + table_name) 
        else:  
            #table isn't there so just insert it
            df_new.write.mode("overwrite").format("delta").save("Tables/" + table_name)

        #delete the files
        if Remove_delta:  
            files = glob.glob(folder_path + entry.name + "/*")  
            for f in files:
                os.remove(f)

StatementMeta(, daf5b681-1577-45ce-aec5-3696d3564aa0, 14, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
